In [ ]:
import pandas as pd 

amazon_data = pd.read_json('/home/adelo/1-system/1-disco_local/1-mis_archivos/1-pe/1-ciencia/1-computer_science_an_IT/2-data_science/1-Amazon_Laptops_Dashboard/amazon_data.json')

amazon_data[['ASIN','price','average_customer_reviews','number_reviews','number_ratings','tech_details','reviews']]

In [ ]:
import pandas as pd

amazon_data = pd.read_json('/home/adelo/1-system/1-disco_local/1-mis_archivos/1-pe/1-ciencia/1-computer_science_an_IT/2-data_science/1-Amazon_Laptops_Dashboard/amazon_data.json')

reviews_title = []
reviews_text  = []
reviews_one_string = []
for i in range(amazon_data.shape[0]):
    reviews_title_per_item = []
    reviews_text_per_item  = []
    reviews_one_string_per_item = ''
    for j in range(len(amazon_data['reviews'][i])):
        title = amazon_data['reviews'][i][j]['title']
        text  = amazon_data['reviews'][i][j]['review_text']
        reviews_title_per_item.append(title)
        reviews_text_per_item.append(text)
        reviews_one_string_per_item += title+' '+text+' '
    reviews_title.append(reviews_title_per_item)
    reviews_text.append(reviews_text_per_item)
    reviews_one_string.append(reviews_one_string_per_item.rstrip())

amazon_data['brand']         = [ amazon_data['tech_details'][i]['Brand Name'].title()  if   amazon_data['tech_details'][i]['Brand Name'] not in ['HP','hp','Hp']  else    amazon_data['tech_details'][i]['Brand Name'].upper()  for  i  in range(amazon_data.shape[0]) ]
amazon_data['series']        = [ amazon_data['tech_details'][i]['Series']              for  i  in range(amazon_data.shape[0]) ]
amazon_data['model_number']  = [ amazon_data['tech_details'][i]['Item model number']   for  i  in range(amazon_data.shape[0]) ]
amazon_data['reviews_title'] = reviews_title
amazon_data['reviews_text']  = reviews_text
amazon_data['reviews_one_string']  = reviews_one_string

def format_cleaner(price):
    return float(price.replace(',','').replace('$',''))

amazon_data['average_customer_reviews'] = [ format_cleaner(val[0]) for val in amazon_data['average_customer_reviews'].str.split() ]

amazon_data['price'] = amazon_data['price'].apply(lambda val: round(format_cleaner(val)) if pd.notnull(val) else val)

In [ ]:
cols = ['brand','series','model_number','price','average_customer_reviews','number_reviews','number_ratings','reviews_title','reviews_text','reviews_one_string','url']
amazon_data = amazon_data[cols]

# display(type(amazon_data['price'][0]))
# display((amazon_data['price'][0]))
# display(type(amazon_data['reviews_one_string'][0]))

# display(amazon_data)

In [ ]:
import string
import nltk
nltk.data.path.append('/home/adelo/.nltk/nltk_data')
from nltk.corpus import stopwords

stopwords_brands = [ b.lower() for b in list(set(amazon_data['brand'])) ]
stopwords_brands_additionals = ['computer','computers','laptop','laptops','thing','things','machine','machines','im','dont','ive']
stopwords_total  = stopwords.words('english') + stopwords_brands + stopwords_brands_additionals

def pre_processing(texto):
    nopunct = ''.join([ char for char in texto if char not in string.punctuation ])
    return ' '.join([ word for word in nopunct.split() if word.lower() not in stopwords_total ])

frase = "Here.. the, sentence! that Has SOME $some the extra processing the the the hola"
frase_clean = pre_processing(frase)
frase_clean

amazon_data['reviews_one_string'] = amazon_data['reviews_one_string'].apply(pre_processing)

In [ ]:
frase = "Here! A sentence: It has $SOME punctuation and stopwords..."
frase_clean = pre_processing(frase)
frase_clean

In [ ]:
display(amazon_data)

In [ ]:
amazon_data.to_json(r'./data.json')

In [5]:
import pandas as pd

data = pd.read_json('./amazon_data.json')

cols = ['brand','series','model_number','price','average_customer_reviews','number_reviews','number_ratings','reviews_one_string']

data[cols]
# type(data['reviews_text'].iloc[1])

,brand,series,model_number,price,average_customer_reviews,number_reviews,number_ratings,reviews_one_string
0,HP,HP Pavilion x360 Convertible 14-cd1042nr,7PD74UA#ABA,690.0,4.5,17,18,everyday persons review friends work rant rave...
1,Lenovo,Flex 14,81SQ000DUS,490.0,4.0,14,18,product COME ADVERTISED sent COMEPLETELY DIFFE...
2,Lenovo,100e Chromebook 2nd Gen,81QB000AUS,99.0,4.1,15,17,Less Great value tough box price excellent Chr...
3,Acer,SF515-51T-73TY,SF515-51T-73TY,927.0,4.0,24,25,Love bought replace old features looking reaso...
4,Asus,Chromebook C425,C425TA-DH384,NaN,3.9,43,48,Experience Asus434 Couldnt Provide give contex...
5,Dell,i5481-3595GRY-PUS,na,400.0,3.5,57,66,BUY NEED ANYTHING MICROSOFT APPS allow downloa...
6,Lenovo,Ideapad S330,81JW0000US,NaN,3.9,88,100,First Chromebook first Chromebook love simplic...
7,HP,HP,NA,357.0,3.7,82,100,waste money Brand new right box slower 10 year...
8,HP,14-cb159nr,8MB46UA#ABA,229.0,3.9,25,29,Mediocre Corporate Style may like one people s...
9,Lenovo,L340 Gaming,81LK00HDUS,625.0,4.1,42,58,Couple days in… far ticked boxes buy looking u...


In [7]:
# https://plotly.com/python/static-image-export/

import plotly.express as px
import plotly.graph_objs as go
fig = px.scatter(data, x="price", y="average_customer_reviews",
                size="average_customer_reviews", color="brand",
                hover_name="brand", log_x=False, size_max=30)

fig.update_layout(
        width=900, height=300,
        margin=go.layout.Margin(l=5, r=5, t=10, b=0), xaxis_tickangle=0,
        uniformtext_minsize=8, uniformtext_mode='hide', coloraxis_showscale=False,
        yaxis=dict(
            title='Avg. customer reviews (1-5)',
            titlefont_size=18,
            tickfont_size=13,
        ),
        xaxis=dict(
            title='Prices ($)',
            titlefont_size=18,
            tickfont_size=15,
        ),
        font=dict(
            size=20,
            color="black"
        ),
        legend_title="Brand",
)
# fig.write_image("images/fig1.pdf")


In [ ]:
# Sentiment Analysis

from textblob import TextBlob
TextBlob('I am great but the impotant thing is that I am not undarstanding what I have to do but I am not loving it ok and it is really bad bad bad bad bad bad bad bad bad bad lov').sentiment

TextBlob(amazon_data['reviews_one_string'].loc[1]).sentiment